In [79]:
import pandas as pd
import numpy as np

In [80]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df1 = pd.read_csv('titanic_train.csv')

In [81]:
train_df = train_df.append(train_df1)

In [82]:
# train_df['Name'] = train_df.Name.map(lambda x: x.split(','))
# train_df['first_n'] = train_df.Name.map(lambda x:x[0])
# train_df['last_n'] = train_df.Name.map(lambda x:x[1])
# test_df['first_n'] = test_df.Name.map(lambda x:x[0])
# test_df['last_n'] = test_df.Name.map(lambda x:x[1])

In [83]:
def data_cleaning(data):
    data = data.drop(['Name','Cabin','Ticket'],axis=1)
    data['Age'] = data['Age'].fillna(0)
    data['Fare'] = data['Fare'].fillna(0)
    data = pd.get_dummies(data)
    return data

In [84]:
train = data_cleaning(train_df)
test = data_cleaning(test_df)

In [85]:
test_id = test.PassengerId
test = test.drop('PassengerId',axis=1)
train = train.drop('PassengerId',axis=1)
y_train = train.Survived
X_train = train.drop('Survived',axis=1)

In [86]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2)
pred = pd.DataFrame()

In [87]:
print(X_train.columns)
print(test.columns)

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male',
       'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')
Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male',
       'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')


In [88]:
%%time
from sklearn.svm import SVC
clf = SVC(C = 2)
clf.fit(X_train, y_train) 
pred['m1'] = clf.predict(test)

CPU times: user 137 ms, sys: 3.31 ms, total: 140 ms
Wall time: 141 ms


In [89]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train) 
pred['m2'] = gnb.predict(test)

In [90]:
from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier(n_estimators=50)
clf1.fit(X_train, y_train) 
pred['m3'] = clf1.predict(test)

In [91]:
from sklearn.ensemble import ExtraTreesClassifier
clf2 = ExtraTreesClassifier(n_estimators=50)
clf2.fit(X_train, y_train) 
pred['m4'] = clf2.predict(test)

In [92]:
from sklearn.ensemble import AdaBoostClassifier
clf3 = AdaBoostClassifier(n_estimators=50)
clf3.fit(X_train, y_train) 
pred['m5'] = clf3.predict(test)

In [93]:
from sklearn.ensemble import GradientBoostingClassifier
clf4 = GradientBoostingClassifier(n_estimators=50)
clf4.fit(X_train, y_train) 
pred['m6'] = clf4.predict(test)

In [94]:
from sklearn import linear_model
clf5 = linear_model.SGDClassifier()
clf5.fit(X_train, y_train) 
pred['m7'] = clf5.predict(test)

/Users/marksonzhang/Library/Python/3.6/lib/python/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [95]:
y_pred = pred.mode(axis=1)

In [96]:
y_pred

,0
0,0
1,0
2,0
3,0
4,1
5,0
6,0
7,0
8,1
9,0


In [97]:
sub = pd.DataFrame({'PassengerId': test_id, 'Survived': y_pred[0].astype('int').values}, columns=['PassengerId', 'Survived'])
sub.to_csv('ensemble.csv', index=False)